### Proyecto Individual Henry
**6_Funciones**  
Funciones para la API
**Autor: Bioing. Urteaga Facundo Nahuel**  

**Resumen:** Este script comprende las siguientes etapas:

1. **Carga de librerías**
2. **Función PlayTimeGenre**
3. **Función UserForGenre**
4. **Función UsersRecommend**
5. **Segundo entrenamiento del modelo (V2)**
6. **Tercer entrenamiento del modelo (V3)**
7. **Cuarto entrenamiento del modelo (V4)**

In [1]:
# 1. Carga de librerías

import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [2]:
# 2. Carga de archivos

df_games_tec = pd.read_parquet('df_games_tec.parquet')
df_games_genres = pd.read_parquet('df_games_genres.parquet')
df_games_specs = pd.read_parquet('df_games_specs.parquet')
df_games_tags = pd.read_parquet('df_games_tags.parquet')
df_reviews_con_sa = pd.read_parquet('df_reviews_con_sa.parquet')
df_items = pd.read_parquet('df_items.parquet')

In [3]:
df_reviews_con_sa["recommend"].head

<bound method NDFrame.head of 0        True
1        True
2        True
3        True
4        True
         ... 
59300    True
59301    True
59302    True
59303    True
59304    True
Name: recommend, Length: 59305, dtype: bool>

In [15]:
# 2. Función PlayTimeGenre

# def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
# Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

Index(['user_id', 'item_id', 'playtime_forever', 'playtime_2weeks'], dtype='object')

In [21]:
def PlayTimeGenre(genero):

    df_games_tec = pd.read_parquet('df_games_tec.parquet')
    df_games_genres = pd.read_parquet('df_games_genres.parquet')
    df_items = pd.read_parquet('df_items.parquet')

    merged_df_1 = pd.merge(df_games_tec[["app_name","item_id","release_year"]], df_games_genres, on='item_id', how='inner')
    merged_df_2 = pd.merge(merged_df_1, df_items[["item_id","playtime_forever"]], on='item_id', how='inner')

    # Lista de géneros disponibles
    generos_disponibles = ['Utilities', 'Racing', 'Massively Multiplayer', 'Sports', 'Action', 
                           'Audio Production', 'Indie', 'Web Publishing', 'RPG', 'Photo Editing', 
                           'Casual', 'Software Training', 'Animation & Modeling', 
                           'Design & Illustration', 'Simulation', 'Adventure', 'Early Access', 
                           'Video Production', 'Education', 'Accounting', 'Free to Play', 'Strategy']
    
    # Verifica si el género especificado existe en la base de datos
    if genero not in merged_df_2.columns:
        return f"No se encontró el género '{genero}' en la base de datos. Géneros disponibles: {', '.join(generos_disponibles)}"
    
    # Filtra el DataFrame por el género especificado
    df_genero_especifico = merged_df_2[merged_df_2[genero] == 1]
    
    # Verifica si no hay datos para el género especificado
    if df_genero_especifico.empty:
        return f"No hay datos para el género '{genero}'. Géneros disponibles: {', '.join(generos_disponibles)}"
    
    # Agrupar por año de lanzamiento y sumar las horas jugadas
    df_horas_jugadas_por_ano = df_genero_especifico.groupby('release_year')['playtime_forever'].sum().reset_index()
    
    # Encontrar el año con la mayor cantidad de horas jugadas acumuladas
    max_total_playtime_year = df_horas_jugadas_por_ano.loc[df_horas_jugadas_por_ano['playtime_forever'].idxmax(), 'release_year']
    
    return f"{{'Año de lanzamiento con más horas jugadas para Género {genero}': {max_total_playtime_year}}}"

# Ejemplo de uso
genero_especifico = 'Racing'  # Inserta aquí el género específico que deseas buscar
resultado = PlayTimeGenre(genero_especifico)
print(resultado)

{'Año de lanzamiento con más horas jugadas para Género Racing': 2015}


In [ ]:
# 3. Función UserForGenre

# def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
# Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [40]:
def UserForGenre(genero):

    # Lista de géneros disponibles
    generos_disponibles = ['Utilities', 'Racing', 'Massively Multiplayer', 'Sports', 'Action', 
                           'Audio Production', 'Indie', 'Web Publishing', 'RPG', 'Photo Editing', 
                           'Casual', 'Software Training', 'Animation & Modeling', 
                           'Design & Illustration', 'Simulation', 'Adventure', 'Early Access', 
                           'Video Production', 'Education', 'Accounting', 'Free to Play', 'Strategy']
    
    
    # Verificar si el género especificado existe en el DataFrame df_games_genres
    if genero not in df_games_genres.columns:
        return f"No se encontró el género '{genero}' en la base de datos. Géneros disponibles: {', '.join(generos_disponibles)}"
    
    # Filtro inicialmente los juegos del genero seleccionado
    df_genre = df_games_genres[df_games_genres[genero] == True]
    
    # Filtro solo jugadores que hayan jugado juegos de este género
    df_func_3_aux = pd.merge(df_items[["user_id","item_id",'playtime_forever']], df_genre[["item_id"]], on='item_id', how='inner')
    df_func_3 = pd.merge(df_func_3_aux, df_games_tec[["item_id",'release_year']], on='item_id', how='inner')

    # Calcula la suma total de playtime_forever para cada usuario
    suma_playtime_por_usuario = df_func_3.groupby('user_id')['playtime_forever'].sum()

    # Encuentra el usuario con la mayor suma de playtime_forever
    usuario_max_playtime = suma_playtime_por_usuario.idxmax()

    # Filtra el DataFrame original para obtener solo las filas asociadas con ese usuario
    df_usuario_max_playtime = df_func_3[df_func_3['user_id'] == usuario_max_playtime]

    # Agrupa las horas jugadas por año para ese usuario
    horas_por_anio = df_usuario_max_playtime.groupby('release_year')['playtime_forever'].sum()

    # Construye el diccionario con el formato deseado
    resultado = {
    f"Usuario con más horas jugadas para Género {genero}": usuario_max_playtime,
    "Horas jugadas": [{"Año": año, "Horas": horas} for año, horas in horas_por_anio.items()]
    }
    
    return resultado

In [ ]:
# 4. Función UsersRecommend
# def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. 
# (reviews.recommend = True y comentarios positivos/neutrales)

In [49]:
def UsersRecommend(anio):

    df_reviews_juegos = pd.merge(df_reviews_con_sa[['item_id', 'recommend','sentiment_analysis']], 
                             df_games_tec[["item_id","app_name","release_year"]], on='item_id', how='inner')
    
    # Verificar si el año especificado está presente en el DataFrame
    if anio not in df_reviews_juegos['release_year'].unique():
        return "Año no encontrado en la base de datos."
    
    # Filtrar el DataFrame por el año especificado
    df_anio = df_reviews_juegos[df_reviews_juegos['release_year'] == anio]
    
    # Filtrar solo las reseñas con recomendación positiva o neutral
    df_recomendados = df_anio[(df_anio['recommend']) & (df_anio['sentiment_analysis'].isin([1, 2]))]
    
    # Contar las recomendaciones por juego
    recomendados_por_juego = df_recomendados.groupby('app_name')['recommend'].sum().reset_index()
    
    # Ordenar los juegos por la cantidad de recomendaciones y obtener el top 3
    top_3_juegos = recomendados_por_juego.nlargest(3, 'recommend')
    
    # Formatear el resultado en el formato de cadena de caracteres requerido
    resultado = [{"Puesto " + str(i+1): juego} for i, juego in enumerate(top_3_juegos['app_name'])]
    
    return resultado

# Ejemplo de uso
anio_dado = 2012  # Inserta aquí el año deseado
resultado = UsersRecommend(anio_dado)
print(resultado)

[{'Puesto 1': 'Counter-Strike: Global Offensive'}, {'Puesto 2': 'Borderlands 2'}, {'Puesto 3': 'PlanetSide 2'}]


In [ ]:
# 4. Función UsersNotRecommend
# def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. 
# (reviews.recommend = False y comentarios negativos)

In [52]:
def UsersNotRecommend(anio):

    df_reviews_juegos = pd.merge(df_reviews_con_sa[['item_id', 'recommend','sentiment_analysis']], 
                             df_games_tec[["item_id","app_name","release_year"]], on='item_id', how='inner')
    
    # Verificar si el año especificado está presente en el DataFrame
    if anio not in df_reviews_juegos['release_year'].unique():
        return "Año no encontrado en la base de datos."
    
    # Filtrar el DataFrame por el año especificado
    df_anio = df_reviews_juegos[df_reviews_juegos['release_year'] == anio]
    
    # Filtrar solo las reseñas con recomendación negativa
    df_no_recomendados = df_anio[(~df_anio['recommend']) & (df_anio['sentiment_analysis'] == 0)]
    
    # Contar las no recomendaciones por juego
    no_recomendados_por_juego = df_no_recomendados.groupby('app_name')['recommend'].count().reset_index()
    
    # Ordenar los juegos por la cantidad de no recomendaciones y obtener el top 3
    top_3_juegos = no_recomendados_por_juego.nlargest(3, 'recommend')
    
    # Formatear el resultado en el formato de cadena de caracteres requerido
    resultado = [{"Puesto " + str(i+1): juego} for i, juego in enumerate(top_3_juegos['app_name'])]
    
    return resultado

# Ejemplo de uso
anio_dado = 2010  # Inserta aquí el año deseado
resultado = UsersNotRecommend(anio_dado)
print(resultado)

[{'Puesto 1': "Assassin's Creed 2 Deluxe Edition"}, {'Puesto 2': 'Poker Night at the Inventory'}, {'Puesto 3': 'Star Trek Online'}]


In [ ]:
# 5. Función Sentiment_Analysis
# def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros 
# de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [54]:
def sentiment_analysis(anio):

    df_reviews_juegos = pd.merge(df_reviews_con_sa[['item_id', 'recommend','sentiment_analysis']], 
                             df_games_tec[["item_id","app_name","release_year"]], on='item_id', how='inner')
    
    # Verificar si el año especificado está presente en el DataFrame
    if anio not in df_reviews_juegos['release_year'].unique():
        return "Año no encontrado en la base de datos."
    
    # Filtrar el DataFrame por el año especificado
    df_anio = df_reviews_juegos[df_reviews_juegos['release_year'] == anio]
    
    # Contar la cantidad de registros para cada categoría de análisis de sentimiento
    conteo_sentimientos = df_anio['sentiment_analysis'].value_counts()
    
    # Formatear el resultado en el formato requerido
    resultado = {
        "Negative": conteo_sentimientos.get(0, 0),
        "Neutral": conteo_sentimientos.get(1, 0),
        "Positive": conteo_sentimientos.get(2, 0)
    }
    
    return resultado

# Ejemplo de uso
anio_dado = 2012  # Inserta aquí el año deseado
resultado = sentiment_analysis(anio_dado)
print(resultado)

{'Negative': 1286, 'Neutral': 3013, 'Positive': 4756}


In [ ]:
# 6. def recomendacion_juego( id de producto ): 
# Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [62]:
def recomendacion_juego(item_id):

    from sklearn.neighbors import NearestNeighbors
    
    df_sist_reco_v4 = pd.read_parquet('df_sist_reco_v4.parquet')

    games_dummies = df_sist_reco_v4.drop(columns=['item_id', 'app_name'])

    n_neighbors=6
    nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(games_dummies)

    # Extraer los vectores de características del juego seleccionado
    game_eval = np.array(games_dummies.loc[item_id]).reshape(1, -1)
    
    # Obtener los índices de los juegos más similares
    dif, ind = nneighbors.kneighbors(game_eval)
    
    # Lista para almacenar los juegos recomendados en formato string
    juegos_recomendados = []
    
    # Juego seleccionado
    juego_seleccionado = {"Juego Seleccionado": df_sist_reco_v4.loc[ind[0][0], 'app_name']}
    juegos_recomendados.append(juego_seleccionado)
    
    # Juegos recomendados
    for i in range(1, 6):  # Tomamos los 5 juegos recomendados
        juego_recomendado = {"Juego Recomendado " + str(i): df_sist_reco_v4.loc[ind[0][i], 'app_name']}
        juegos_recomendados.append(juego_recomendado)
    
    return juegos_recomendados

# Ejemplo de uso
item_id_dado = 32103  # Inserta aquí el item_id deseado
resultado = recomendacion_juego(item_id_dado)
print(resultado)

[{'Juego Seleccionado': 'Flappy Arms'}, {'Juego Recomendado 1': 'Shooty Fruity'}, {'Juego Recomendado 2': 'Guided Meditation VR'}, {'Juego Recomendado 3': 'EXA: The Infinite Instrument'}, {'Juego Recomendado 4': 'Universe Sandbox ²'}, {'Juego Recomendado 5': 'Salvage Op'}]


C:\Users\Facundo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
